#### A quick demonstration of bias-hardened lensing/patchy tau/point source reconstruction in fullsky
#### - this code shows how to compute the estiamtors, and output their power spectra

In [1]:
import numpy as np, basic, curvedsky as cs, plottools as pl, pickle, quad_func, tqdm, misctools, cmb
from matplotlib.pyplot import *

In [2]:
# define parameters
Tcmb  = 2.726e6    # CMB temperature
lmax  = 2048       # maximum multipole of output normalization
lmax  = 512
rlmin, rlmax = 100, lmax  # reconstruction multipole range
sig   = 50.
ac2rad = np.pi/180./60.
L = np.linspace(0,lmax,lmax+1)
simn = 100
root = '/global/homes/t/toshiyan/scratch/test/'
ow = False

#### load unlensed and lensed Cls

In [3]:
lcl = cmb.read_camb_cls('../data/lensedcls.dat',ftype='lens',output='array')[:,:lmax+1]
nl  = np.zeros((4,lmax+1))
nl[0,:] = (sig*ac2rad/Tcmb)**2
nl[1,:] = 2*nl[0,:]
nl[2,:] = 2*nl[0,:]
ocl = lcl + nl

#### Calculate normalizations, N0 and RDN0

In [4]:
# example setup of filename and its ids
ids = [str(i).zfill(5) for i in range(-1,1000)]
ids[0] = 'real'  # change 1st index
falm = { m: [root+'tau/cmb/test_'+m+'_'+x+'.pkl' for x in ids] for m in ['T','E','B'] }
fcgg = { q: [root+'tau/aps/cgg_'+q+'_'+x+'.txt' for x in ids] for q in ['TT'] }

In [5]:
# generate CMB alms 
for i in tqdm.tqdm(range(simn+1)):
    if misctools.check_path(falm['T'][i]): continue
    Talm = cs.utils.gauss1alm(lmax,lcl[0,:])
    Talm += cs.utils.gauss1alm(lmax,nl[0,:])
    pickle.dump((Talm),open(falm['T'][i],"wb"),protocol=pickle.HIGHEST_PROTOCOL)

100%|██████████| 101/101 [00:03<00:00, 27.76it/s]


In [6]:
qobj = quad_func.quad(root=root,rlz=range(simn+1),qtype='tau',bhe=['lens'],\
        lcl=lcl,ocl=ocl,ifl=ocl,falm=falm,rlmin=rlmin,rlmax=rlmax,n0max=np.int(simn/2),rdmax=simn,nside=1024,qlist=['TT'],ids=ids,overwrite=ow)

/global/homes/t/toshiyan/.conda/envs/loc/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [7]:
qobj.qrec_flow(run=['norm','qrec','n0','rdn0'])
#qobj.qrec_flow(run=['drdn0'])

RDN0:: 100%|██████████████████████████████████████████████████████| 101/101 [35:25<00:00, 21.05s/it]


In [8]:
rds = np.array([(np.loadtxt(qobj.f['TT'].rdn0[i+1])).T[1] for i in range(simn)])

OSError: /global/homes/t/toshiyan/scratch/test/tau/rdn0/rdn0_TT__bh-lens_l100-512_n100_00000.dat not found.

In [ ]:
xlim(1,rlmax)
ylim(1e-8,1e-4)
yscale('log')
plot(l,np.std(rds,axis=0))
plot(l,n0/np.sqrt(l+0.5))

In [ ]:
cl = np.zeros((simn,qobj.olmax+1))
for i in tqdm.tqdm(range(simn)):
    #if misctools.check_path(fcgg['TT'][i+1],overwrite=ow): 
    #    cl[i,:] = (np.loadtxt(fcgg['TT'][i+1])).T
    #else:
        glm = pickle.load(open(qobj.f['TT'].alm[i+1],"rb"))[0]
        cl[i,:] = cs.utils.alm2cl(qobj.olmax,glm)
        np.savetxt(fcgg['TT'][i+1],cl[i,:].T)

In [ ]:
xlim(1,rlmax)
ylim(1e-7,1e-3)
yscale('log')
#plot(l,np.mean(cl,axis=0))
#plot(l,n0)
#plot(l,al)
plot(l,np.std(cl,axis=0))
plot(l,np.std(cl-rds,axis=0))